<a href="https://colab.research.google.com/github/onishchenkoar/dungeons-and-dragons-and-probability/blob/main/Critical_Role_E66_A_Traveler's_Gamble_House_edge_for_the_2d6_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Critical Role E66: A Traveler's Gamble. House edge for the 2d6 game

In episode 66 of Critical Role, [there is a dice game](https://youtu.be/jgmBV5NA2A8?t=8347) called Avandra's Favor: make a bet, roll 2d6; if you have 7 or 12 &mdash; you win, if you have anything else &mdash; you either lose the bet or can double the bet and roll an additional d6 &mdash; for the same winning numbers. Below is a code snippet that finds the best betting strategy and calculates [house edge](https://en.wikipedia.org/wiki/Gambling_mathematics#House_advantage_or_edge) of the game.

In [5]:
import itertools

import numpy as np

WINNING_NUMBERS = {7, 12}

# Build a probability distribution of 2d6 roll:
dist2d6 = np.convolve([1/6]*6, [1/6]*6)
dist2d6 = np.concatenate(([0, 0], dist2d6))
values_after_1st_round = set(range(2, 13)) - WINNING_NUMBERS

# Find conditional probabilities of success in the second round:
second_round_conditionals = np.zeros_like(dist2d6)
for value in values_after_1st_round:
  for i in range(1, 7):
    if value + i in WINNING_NUMBERS:
      second_round_conditionals[value] += 1
second_round_conditionals = second_round_conditionals / 6

p_win_1st_round = sum(x for x in dist2d6[list(WINNING_NUMBERS)])
best_comb = []
best_expectation = p_win_1st_round - (1 - p_win_1st_round)
worst_comb = []
worst_expectation = p_win_1st_round - (1 - p_win_1st_round)
consistency_check = []

# Exhaustive search for numbers you should double down after the first round:
for r in range(1, len(values_after_1st_round) + 1):
  for raise_on in itertools.combinations(values_after_1st_round, r):
    dont_raise_on = list(values_after_1st_round - set(raise_on))
    raise_on = list(raise_on)
    p_lose_1st_round = sum(dist2d6[dont_raise_on])
    p_win_2nd_round = np.dot(second_round_conditionals[raise_on],
                              dist2d6[raise_on])
    p_lose_2nd_round = np.dot(1 - second_round_conditionals[raise_on],
                              dist2d6[raise_on])
    
    expectation = p_win_1st_round + 2*p_win_2nd_round \
                  - p_lose_1st_round - 2*p_lose_2nd_round
    if expectation > best_expectation:
      best_comb = raise_on
      best_expectation = expectation

    if expectation < worst_expectation:
      worst_comb = raise_on
      worst_expectation = expectation
    
    consistency_check.append(abs(p_win_1st_round
                                 + p_lose_1st_round
                                 + p_win_2nd_round
                                 + p_lose_2nd_round - 1) < 1e-12)

print('Sum of probabilities == 1:', all(consistency_check))
print()
print('Best for player:', best_comb, best_expectation)
print('Worst for player:', worst_comb, worst_expectation)

Sum of probabilities == 1: True

Best for player: [6] -0.5648148148148148
Worst for player: [2, 3, 4, 5, 8, 9, 10, 11] -0.8333333333333334


This result means, that the best strategy for a player would be to raise the bet only when he has 6 in the first round, and the worst is to raise the bet on any other number. This game is terrible for the player: with the best strategy house edge is 56% (83% with the worst). This means that, in the long run, casino will pocket 56% of player's money. For example, [here](https://wizardofodds.com/gambling/house-edge/) is a list of games with their correspondent house edge. 33.33% is the highest house edge on the list. Avandra's Favor is, basically, a robbery.

## Can this game be better?
One simple way to improve the odds of the player is to add some more winning numbers. By changing `WINNING_NUMBERS` in the code, I have found that if a player wins on 3, 7, 9, 12, the house edge is 1.8%, which has at least an illusion of fairness.